<a href="https://colab.research.google.com/github/cisimon7/Optimization-Technics-and-Application/blob/main/Trajectory%20Optimization/Cart_Pole/Direct%20Method%20(python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>Trajectory Optimization of Cart-Pole System</b></h1>
<h2 align="center"><b>Simon Idoko</b></h2>
<h3><i>A Test of Trajectory Optimization on a Cart-Pole System before using on main thesis project</i></h3>

###<b>Imports</b>

In [ ]:
from IPython.display import clear_output 
!pip install casadi
clear_output()

In [ ]:
import numpy as np
import casadi as csd
from functools import reduce
from casadi import MX, SX, DM, Function
np.set_printoptions(linewidth=200, precision=4, suppress=True)

In [ ]:
import plotly.graph_objects as go

###<b>Parameter Initialization</b>

In [ ]:
# SYSTEM DIMENSIONS
m1   = 1                    # Mass of Cart  
m2   = 0.3                  # Mass of Pole 
L    = 0.5                  # Length of rod 
g    = 9.81                 # Gravity       

In [ ]:
# COLLOCATION DETAILS
tmin  = 0                   # Time start
tmax  = 5                   # Time End
n     = 40                  # Number of grid points
h     = (tmax-tmin)/(n)     # Time Step

In [ ]:
# PATH LENGTH BOUNDARIES
dmin  = -2                  # Start of finite path
dmax  = +2                  # End of finit path

In [ ]:
# CONTROL TORQUE BOUNDARIES
umin  = -20                 # Minimum torque
umax  = +20                 # Maximum Torque

In [ ]:
# INITIAL STATE
d0     = -0.5                 # Desired Final position
theta0 = 0                    # Desired Angle

In [ ]:
# DESIRED FINAL STATE
d     = 0.5                 # Desired Final position
theta = np.pi               # Desired Angle

In [ ]:
params = [ m1, m2, L, g ]

###<b>Dynamics, Objectives and Constraints</b>


#####<i>States Definition</i>


In [ ]:
# STATES AND PARAMETERS DEFINITION
opti = csd.Opti()                                   # CasADi Optimization Object

time = np.arange(tmin,tmax,h)                       # Discretized Time Sequence
us   = opti.variable(n)                             # u - representing Controller torque

q1s, q2s     = opti.variable(n), opti.variable(n)   # Cart Pole Position and Pole Angle [   q1,   q2 ]  
qs           = csd.horzcat(q1s,q2s)  

qd1s, qd2s   = opti.variable(n), opti.variable(n)   # Cart Pole Position and Pole Angle [ qdd1, qdd2 ] 
qds          = csd.horzcat(qd1s,qd2s)  

qdd1s, qdd2s = opti.variable(n), opti.variable(n)   # Cart Pole Position and Pole Angle [   q1,   q2 ]  
qdds         = csd.horzcat(qdd1s,qdd2s)  

states  = csd.horzcat(qs,qds)                       # [[  q1,  q2,  qd1,  qd2 ]]
dstates = csd.horzcat(qds,qdds)                     # [[ qd1, qd2, qdd1, qdd2 ]]

In [ ]:
print(states.shape)
print(dstates.shape)

(40, 4)
(40, 4)


#####<i>Dynamics Equation in Standard Manipulator Form</i>


In [ ]:
def Inertia(t_k, state_k):
  q1, q2, qd1, qd2 = state_k[0], state_k[1], state_k[2], state_k[3]
  
  return csd.vertcat(
      csd.horzcat(            m1+m2,  m1*L*csd.cos(q2) ),
      csd.horzcat( m1*L*csd.cos(q2),          m1*L**2  )
  )

In [ ]:
def Coriolis(t_k, state_k):
  q1, q2, qd1, qd2 = state_k[0], state_k[1], state_k[2], state_k[3]

  return csd.vertcat(
      csd.horzcat( 0, -m1*qd2*csd.sin(q2) ),
      csd.horzcat( 0,                   0 )
   )

In [ ]:
def Gravity(t_k, state_k):
  q1, q2, qd1, qd2 = state_k[0], state_k[1], state_k[2], state_k[3]

  return csd.vertcat( 0, m2*g*L*csd.sin(q2) )

In [ ]:
def TorqueMat(t_k, state_k):
  q1, q2, qd1, qd2 = state_k[0], state_k[1], state_k[2], state_k[3]

  return csd.vertcat( 1, 0 )  

In [ ]:
def Dynamics(t_k, state_k, controller_K):
  M = Inertia(t_k, state_k)
  C = Coriolis(t_k, state_k)
  G = Gravity(t_k, state_k)
  B = TorqueMat(t_k, state_k)
  u = controller_K

  qdd = csd.inv(M) @ (B@u - G - C@(state_k[2:4].T))

  return csd.horzcat(state_k[2:4], qdd.T)

#####<i>Cost Function</i>


In [ ]:
"""The Cost function we use is the total energy (sum of Kinetic and Potential Energy) in the system"""
def Cost(t_k,state_k,controller_k):  
  q1, q2, qd1, qd2 = state_k[0], state_k[1], state_k[2], state_k[3]

  KE = (0.5 * (m1+m2) * qd1**2) + m1*qd1*qd2*L*csd.cos(q2) + (0.5 * m1 * L**2 * qd2**2)
  # PE = m2*g*L*csd.cos(q2)

  energy = KE 

  return energy

In [ ]:
w = [ Cost(time[i],states[i,:],us[i]) for i in range(n) ]
objective = reduce(lambda w1,w2: w1+w2, [ h/2 * (w[i] + w[i+1]) for i in range(n-1) ])

#####<i>Controller Constraints</i>


In [ ]:
control_constraints = [ opti.bounded(umin,us[i],umax) for i in range(n) ]

#####<i>Path Constraints</i>


In [ ]:
path_constraints = [ opti.bounded(dmin,q1s[i],dmax) for i in range(n) ] # Keep Cart on finite Track Length

#####<i>Boundary Conditions</i>


In [ ]:
state_init  = [ d0, theta0, 0, 0 ]
state_end   = [ d, theta, 0, 0 ]

In [ ]:
boundary_constraints = [
    *[ states[0,i]   == state_init[i] for i in range(4) ],
    *[ states[n-1,i] == state_end[i]  for i in range(4) ]
] 

#####<i>Dynamics Constraints</i>


In [ ]:
def Dynamics_discrete(k):
  dynamics = (h/2) * (Dynamics(time[k], states[k,:], us[k]) + Dynamics(time[k-1], states[k-1,:], us[k-1]))
  return [ states[k,i] - states[k-1,i] == dynamics[i] for i in range(4) ]

In [ ]:
dynamics_constraints = [ cons for k in range(1,n) for cons in Dynamics_discrete(k) ] 

#####<b>Constraints</b>


In [ ]:
constraints = [ *boundary_constraints, *control_constraints, *path_constraints, *dynamics_constraints ]

#####<b>Initial Guess</b>


In [ ]:
init_guess = np.asarray([ (i/n)*np.array([ d, theta, 0, 0 ]) for i in range(states.shape[0]) ])

###<b>Initialize Problem</b>

In [ ]:
opti.minimize(objective)
opti.subject_to(constraints)
for i in range(states.shape[0]):
  for j in range(states.shape[1]):
    opti.set_initial(states[i,j], init_guess[i,j])
opti.solver('ipopt')
sol = opti.solve()
clear_output()

In [ ]:
u_vals    = [ sol.value(us[i])   for i in range(n) ]
q1_vals   = [ sol.value(q1s[i])  for i in range(n) ]
q2_vals   = [ sol.value(q2s[i])  for i in range(n) ]
qd1_vals  = [ sol.value(qd1s[i]) for i in range(n) ]
qd2_vals  = [ sol.value(qd2s[i]) for i in range(n) ]

###<b>Plots of Result</b>

In [ ]:
go.Figure(
  data=[
      go.Scatter(x=time,y=q1_vals,mode='lines+markers',name='Pole Horizontal Position'),
      go.Scatter(x=time,y=init_guess[:,0],mode='lines+markers',name='Guess')
  ],
  layout=go.Layout(width=1000, height=700, showlegend=True, title_text='Cart Pole Position', legend=dict(orientation='h'))
).show()

In [ ]:
go.Figure(
  data=[
      go.Scatter(x=time,y=q2_vals,mode='lines+markers',name='Pole angle'),
      go.Scatter(x=time,y=init_guess[:,1],mode='lines+markers',name='Guess')
  ],
  layout=go.Layout(width=1000, height=700, showlegend=True, title_text='Pole Angle Position', legend=dict(orientation='h'))
).show()

In [ ]:
go.Figure(
  data=[
        go.Scatter(x=time,y=u_vals,mode='lines+markers',name='Control torque')
  ],
  layout=go.Layout(width=1000, height=700, showlegend=True, title_text='Control torque', legend=dict(orientation='h'))
).show()

###<b>Interpolation</b>

In [ ]:
from scipy import interpolate
space = np.linspace(time[0],time[-1],500)

In [ ]:
cspline_u = interpolate.CubicSpline(time, u_vals, bc_type='clamped')
go.Figure(
  data=[
    go.Scatter(x=time,y=u_vals,mode="markers", name='Control Torque Grid Points'),
    go.Scatter(x=space,y=cspline_u(space),mode="lines", name='Interpolated Control Torque')
  ],
  layout=go.Layout(width=1000, height=700, showlegend=True, title_text='Control torque Interpolated', legend=dict(orientation='h'))
).show()

In [ ]:
cspline_q1 = interpolate.CubicSpline(time, q1_vals, bc_type='clamped')
go.Figure(
  data=[
    go.Scatter(x=time,y=q1_vals,mode="markers"),
    go.Scatter(x=space,y=cspline_q1(space),mode="lines")
  ],
  layout=go.Layout(width=1000, height=700, showlegend=False)
).show()

In [ ]:
cspline_q2 = interpolate.CubicSpline(time, q2_vals, bc_type='clamped')
go.Figure(
  data=[
    go.Scatter(x=time,y=q2_vals,mode="markers"),
    go.Scatter(x=space,y=cspline_q2(space),mode="lines")
  ],
  layout=go.Layout(width=1000, height=700, showlegend=False)
).show()

###<b>References</b>

An Introduction to Trajectory Optimization by MATTHEW KELLY